# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import nltk
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger', 'stopwords'])

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
import pandas as pd
import numpy as np
import re
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, precision_recall_fscore_support, f1_score, recall_score, precision_score
from sklearn.model_selection import train_test_split
import pickle
from sqlalchemy import create_engine

In [3]:
# load data from database
engine = create_engine('sqlite:///disaster_messages.db')
df = pd.read_sql_table('messages_categories', engine)
display(df)
X = df['message'] # What we get from the user
Y = df[df.columns[4:]] # what we try to predict

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,14,Information about the National Palace-,Informtion au nivaux palais nationl,direct,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,15,Storm at sacred heart of jesus,Cyclone Coeur sacr de jesus,direct,1,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
7,16,"Please, we need tents and water. We are in Sil...",Tanpri nou bezwen tant avek dlo nou zon silo m...,direct,1,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
8,17,"I would like to receive the messages, thank you",Mwen ta renmen jouin messag yo. Merci,direct,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,18,I am in Croix-des-Bouquets. We have health iss...,"Nou kwadebouke, nou gen pwoblem sant m yo nan ...",direct,1,1,0,1,1,1,...,0,0,0,0,0,0,0,0,0,1


In [4]:
X.head()

0    Weather update - a cold front from Cuba that c...
1              Is the Hurricane over or is it not over
2                      Looking for someone but no name
3    UN reports Leogane 80-90 destroyed. Only Hospi...
4    says: west side of Haiti, rest of the country ...
Name: message, dtype: object

In [5]:
Y.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 2. Write a tokenization function to process your text data

In [6]:
def tokenize(text):
    """ Tokenize a text into a list of lemmatized words where stop words are removed. 
        Also the tokens needs to be stripped from whitespaces.
    
        inputs:
            text: string
        outputs:
            list of lemmatized words
    """
    # First lower case and normalize by removing all the punctuation marks.
    text = re.sub(f"[^a-zA-Z0-9]", " ", text.lower())
    tokens = word_tokenize(text)
    stopwords_to_remove = stopwords.words('english')
    
    lemmatizer = WordNetLemmatizer()
    final_tokens = [lemmatizer.lemmatize(token).strip() for token in tokens if token not in stopwords_to_remove]
    
    return final_tokens

In [7]:
tokenize("I am pleased to announce that I am the best Data Scientist out there!")

['pleased', 'announce', 'best', 'data', 'scientist']

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [8]:
randomforestpipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('classifier', MultiOutputClassifier(RandomForestClassifier(),n_jobs=1))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, Y)
randomforestpipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

In [11]:
randomforestpipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x7db3c5c81b70>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('classifier',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
               max_depth=None, max_features='auto', max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=2,
               min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
               oob_score=False, random_stat

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [12]:
y_pred = randomforestpipeline.predict(X_test)

In [13]:
def display_results(y_pred, y_test, verbose = 1):
    overall_f1 = []
    overall_recall = []
    overall_precision = []
    for idx, column in enumerate(y_test.columns.tolist()):
        if verbose == 1:
            print("Classification Report for column: " + column + "\n")
            print(classification_report(y_test.values[:, idx], y_pred[:, idx]))
        overall_precision.append(precision_score(y_test.values[:, idx], y_pred[:, idx], average='weighted'))
        overall_f1.append(f1_score(y_test.values[:, idx], y_pred[:, idx], average='weighted'))
        overall_recall.append(recall_score(y_test.values[:, idx], y_pred[:, idx], average='weighted'))
    
    if verbose == 1:
        print('Overall precision: ', np.mean(overall_precision))
        print('Overall f1 score: ', np.mean(overall_f1))
        print('Overall recall: ', np.mean(overall_recall))
    
    return overall_precision, overall_f1, overall_recall

In [14]:
overall_precision, overall_f1, overall_recall = display_results(y_pred, y_test)

Classification Report for column: related

             precision    recall  f1-score   support

          0       0.62      0.43      0.51      1589
          1       0.83      0.92      0.87      4965

avg / total       0.78      0.80      0.78      6554

Classification Report for column: request

             precision    recall  f1-score   support

          0       0.90      0.98      0.94      5479
          1       0.80      0.43      0.56      1075

avg / total       0.88      0.89      0.88      6554

Classification Report for column: offer

             precision    recall  f1-score   support

          0       1.00      1.00      1.00      6525
          1       0.00      0.00      0.00        29

avg / total       0.99      1.00      0.99      6554

Classification Report for column: aid_related

             precision    recall  f1-score   support

          0       0.76      0.86      0.81      3909
          1       0.74      0.60      0.66      2645

avg / total       0.

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [16]:
parameters = {
    'classifier__estimator__n_estimators': [10, 30, 50, 100],
    'classifier__estimator__max_depth': [10, 50],
    'classifier__estimator__random_state': [42]
}

randomForestCv = GridSearchCV(randomforestpipeline, param_grid=parameters)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [22]:
randomForestCv.fit(X_train, y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'classifier__estimator__n_estimators': [10, 30, 50, 100], 'classifier__estimator__max_depth': [10, 50], 'classifier__estimator__random_state': [42]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [23]:
y_pred = randomForestCv.predict(X_test)
overall_precision, overall_f1, overall_recall = display_results(y_pred, y_test)


Classification Report for column: related

             precision    recall  f1-score   support

          0       0.00      0.00      0.00      1565
          1       0.76      1.00      0.86      4989

avg / total       0.58      0.76      0.66      6554

Classification Report for column: request

             precision    recall  f1-score   support

          0       0.83      1.00      0.91      5470
          1       0.00      0.00      0.00      1084

avg / total       0.70      0.83      0.76      6554

Classification Report for column: offer

             precision    recall  f1-score   support

          0       1.00      1.00      1.00      6529
          1       0.00      0.00      0.00        25

avg / total       0.99      1.00      0.99      6554

Classification Report for column: aid_related

             precision    recall  f1-score   support

          0       0.61      1.00      0.76      3903
          1       0.94      0.07      0.13      2651

avg / total       0.

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### Comparison
- The standard Random Forest classifier performed better than the one in the gridsearch.

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### AdaBoostClassifier
* Lets try to improve the model performance by trying the AdaBoostClassifier to increase the performance of the model.
* We will also run a gridsearch to ensure we use the best parameters for both models.

In [17]:
adaBoostPipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('classifier', MultiOutputClassifier(AdaBoostClassifier(),n_jobs=1))
])

adaBoostPipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...mator=None,
          learning_rate=1.0, n_estimators=50, random_state=None),
           n_jobs=1))])

In [18]:
y_pred = adaBoostPipeline.predict(X_test)
overall_precision, overall_f1, overall_recall = display_results(y_pred, y_test)


Classification Report for column: related

             precision    recall  f1-score   support

          0       0.67      0.21      0.32      1589
          1       0.79      0.97      0.87      4965

avg / total       0.76      0.78      0.74      6554

Classification Report for column: request

             precision    recall  f1-score   support

          0       0.91      0.97      0.94      5479
          1       0.74      0.49      0.59      1075

avg / total       0.88      0.89      0.88      6554

Classification Report for column: offer

             precision    recall  f1-score   support

          0       1.00      1.00      1.00      6525
          1       0.00      0.00      0.00        29

avg / total       0.99      0.99      0.99      6554

Classification Report for column: aid_related

             precision    recall  f1-score   support

          0       0.77      0.86      0.81      3909
          1       0.75      0.61      0.68      2645

avg / total       0.

- The AdaBoostClassifier standard performs better than the RandomForestClassifier, but lets to a grid search and see.

In [26]:
parameters = {'classifier__estimator__n_estimators':[20, 30, 50, 100], 
              'classifier__estimator__learning_rate':[0.1, 0.2, 0.5, 1.0], 
              'classifier__estimator__algorithm': ('SAMME', 'SAMME.R'),
              'classifier__estimator__random_state': [42]}

adaBoostCv = GridSearchCV(adaBoostPipeline, param_grid=parameters)

In [27]:
cv.fit(X_train, y_train)

KeyboardInterrupt: 

In [ ]:
y_pred = cv.predict(X_test)
overall_precision, overall_f1, overall_recall = display_results(y_pred, y_test)


### 9. Export your model as a pickle file

In [ ]:
best_model = pipeline.best_estimator_
best_model

In [ ]:
with open('adaboostclassifier.pkl', 'wb') as pickle_file:
    pickle.dump(best_model, pickle_file)

### 10. Use this notebook to complete `train_classifier.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.